# Импорт библиотек

In [ ]:
# !python -m spacy download ru_core_news_lg > null

In [71]:
import numpy as np
import evaluate
import collections

from datasets import load_dataset
from transformers import AutoModel
from transformers import AutoTokenizer
from transformers import AutoModelForQuestionAnswering
from transformers import TrainingArguments
from transformers import Trainer

import optuna

from tqdm.auto import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import spacy
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score

# Загрузка данных и установка параметров

In [2]:
# Загрузка датасета
dataset = load_dataset("sberquad")

# Извлечение контекста и вопроса из тренировочного набора данных
train_context = dataset["train"][0]["context"]
train_question = dataset["train"][0]["question"]

In [3]:
train_context

'В протерозойских отложениях органические остатки встречаются намного чаще, чем в архейских. Они представлены известковыми выделениями сине-зелёных водорослей, ходами червей, остатками кишечнополостных. Кроме известковых водорослей, к числу древнейших растительных остатков относятся скопления графито-углистого вещества, образовавшегося в результате разложения Corycium enigmaticum. В кремнистых сланцах железорудной формации Канады найдены нитевидные водоросли, грибные нити и формы, близкие современным кокколитофоридам. В железистых кварцитах Северной Америки и Сибири обнаружены железистые продукты жизнедеятельности бактерий.'

In [4]:
# Определение устройства для выполнения: CUDA (GPU) или CPU
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cuda


In [5]:
# Путь к предварительно обученной модели RuBERT
bert_checkpoint = 'DeepPavlov/rubert-base-cased'

# Инициализация токенизатора для RuBERT
tokenizer = AutoTokenizer.from_pretrained(bert_checkpoint)

In [6]:
# Токенизация вопроса и контекста
tokenized_inputs = tokenizer(train_question, train_context)

# Декодирование токенизированных входных данных обратно в текст для проверки
tokenizer.decode(tokenized_inputs["input_ids"])

# Установка максимальной длины входного текста и шага для разбиения на части (stride)
max_input_length = 384
input_stride = 128

# Предобработка данных

In [7]:
# Функция для предобработки тренировочных данных
def preprocess_train_data(examples):
    # Убираем пробельные символы из вопросов
    question_list = [q.strip() for q in examples['question']]
    
    # Токенизация вопросов и контекстов
    tokenized_data = tokenizer(
        question_list,
        examples['context'],
        max_length=max_input_length,
        truncation='only_second',
        stride=input_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding='max_length',
    )
    
    # Извлекаем данные смещения и данные сэмплирования
    offset_mapping_data = tokenized_data.pop('offset_mapping')
    sample_mapping_data = tokenized_data.pop('overflow_to_sample_mapping')
    
    # Извлекаем данные ответов
    answer_data = examples['answers']
    start_positions_data = []
    end_positions_data = []

    # Определение позиций начала и конца ответа в токенизированных данных
    for (i, offset) in enumerate(offset_mapping_data):
        sample_idx = sample_mapping_data[i]
        answer_info = answer_data[sample_idx]
        start_char_idx = answer_info['answer_start'][0]
        end_char_idx = answer_info['answer_start'][0] + len(answer_info['text'][0])
        sequence_ids_data = tokenized_data.sequence_ids(i)

        # Находим начало и конец контекста в токенизированных данных
        idx = 0
        while sequence_ids_data[idx] != 1:
            idx += 1
        context_start_idx = idx
        while sequence_ids_data[idx] == 1:
            idx += 1
        context_end_idx = idx - 1

        # Определяем позиции начала и конца ответа
        if offset[context_start_idx][0] > start_char_idx \
            or offset[context_end_idx][1] < end_char_idx:
            start_positions_data.append(0)
            end_positions_data.append(0)
        else:
            idx = context_start_idx
            while idx <= context_end_idx and offset[idx][0] <= start_char_idx:
                idx += 1
            start_positions_data.append(idx - 1)

            idx = context_end_idx
            while idx >= context_start_idx and offset[idx][1] >= end_char_idx:
                idx -= 1
            end_positions_data.append(idx + 1)

    # Добавляем позиции начала и конца ответа в токенизированные данные
    tokenized_data['start_positions'] = start_positions_data
    tokenized_data['end_positions'] = end_positions_data
    return tokenized_data

In [8]:
# Функция для предобработки валидационных данных
def preprocess_val_data(examples):
    # Убираем пробельные символы из вопросов
    question_list = [q.strip() for q in examples['question']]
    
    # Токенизация вопросов и контекстов
    tokenized_data = tokenizer(
        question_list,
        examples['context'],
        max_length=max_input_length,
        truncation='only_second',
        stride=input_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding='max_length',
    )

    # Извлекаем данные сэмплирования
    sample_mapping_data = tokenized_data.pop('overflow_to_sample_mapping')
    example_id_data = []

    # Присваиваем каждому токенизированному элементу его идентификатор из исходных данных
    for i in range(len(tokenized_data['input_ids'])):
        sample_idx = sample_mapping_data[i]
        example_id_data.append(examples['id'][sample_idx])

        # Корректировка данных смещения для учета только контекста (без вопроса)
        sequence_ids_data = tokenized_data.sequence_ids(i)
        offset_data = tokenized_data['offset_mapping'][i]
        tokenized_data['offset_mapping'][i] = [(o if sequence_ids_data[k] == 1 else None) for (k, o) in enumerate(offset_data)]

    tokenized_data['example_id'] = example_id_data
    return tokenized_data

In [9]:
# Применяем функции предобработки к наборам данных
processed_train_dataset = dataset["train"].map(
    preprocess_train_data,
    batched=True,
    remove_columns=dataset["train"].column_names,
)

processed_val_dataset = dataset["validation"].map(
    preprocess_val_data,
    batched=True,
    remove_columns=dataset["validation"].column_names,
)

Map:   0%|          | 0/5036 [00:00<?, ? examples/s]

In [10]:
# Выводим размеры наборов данных до и после предобработки
print(f'Размер тренировочного набора до обработки: {len(dataset["train"])}, после: {len(processed_train_dataset)}')
print(f'Размер валидационного набора до обработки: {len(dataset["validation"])}, после: {len(processed_val_dataset)}')

Размер тренировочного набора до обработки: 45328, после: 45544
Размер валидационного набора до обработки: 5036, после: 5063


# Построение и обучение модели

In [11]:
# Загрузка модели для ответа на вопросы из предварительно обученного чекпоинта
qa_model = AutoModelForQuestionAnswering.from_pretrained(bert_checkpoint)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
# Установка параметров обучения
training_args = TrainingArguments(
    "DeepPavlov/rubert-base-cased",    # путь для сохранения обученной модели
    evaluation_strategy="no",          # стратегия оценки во время обучения (в данном случае - нет оценки)
    save_strategy="epoch",             # сохранять модель после каждой эпохи
    learning_rate=2e-5,                # скорость обучения
    num_train_epochs=3,                # количество эпох обучения
    weight_decay=0.01,                 # коэффициент уменьшения весов (L2 регуляризация)
    fp16=True,                         # использование 16-битной точности (уменьшает использование памяти и ускоряет обучение)
)

# Инициализация тренера для обучения модели
trainer_instance = Trainer(
    model=qa_model,                                 # модель для обучения
    args=training_args,                             # параметры обучения
    train_dataset=processed_train_dataset,          # тренировочный набор данных
    eval_dataset=processed_val_dataset,             # валидационный набор данных
    tokenizer=tokenizer,                            # токенизатор для преобразования текста в токены
)

In [13]:
# Запуск процесса обучения
trainer_instance.train()

# Получение предсказаний модели на валидационном наборе данных
predictions, _, _ = trainer_instance.predict(processed_val_dataset)

# Разделение предсказаний на начальные и конечные логиты (вероятности начала и конца ответа)
start_logits, end_logits = predictions

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,2.734100
1000,1.978200
1500,1.861500
2000,1.771400
2500,1.710800
3000,1.659600
3500,1.644400
4000,1.604100
4500,1.600100
5000,1.598000


# Оценка качества модели

In [14]:
# Загрузка метрики для оценки качества модели на задаче SQuAD
evaluation_metric = evaluate.load("squad")

# Установка параметров для вычисления метрик
n_best = 20
max_answer_length = 30

In [15]:
# Функция для вычисления метрик качества модели
def evaluate_model(start_logits_data, end_logits_data, features_data, examples_data):
    # Словарь для хранения связи между примерами и их признаками
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features_data):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples_data):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Обработка всех признаков, связанных с данным примером
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits_data[feature_index]
            end_logit = end_logits_data[feature_index]
            offsets = features_data[feature_index]["offset_mapping"]

            # Выбор лучших индексов начала и конца ответа
            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Пропускаем ответы, которые не полностью находятся в контексте
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Пропускаем ответы слишком короткой или длинной длины
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    # Формирование ответа
                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Выбор наилучшего ответа
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": str(example_id), "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": str(example_id), "prediction_text": ""})

    # Сравнение предсказанных ответов с эталонными
    theoretical_answers = [{"id": str(ex["id"]), "answers": ex["answers"]} for ex in examples_data]
    return evaluation_metric.compute(predictions=predicted_answers, references=theoretical_answers)

# Применение функции оценки качества к предсказаниям модели
evaluate_model(start_logits, end_logits, processed_val_dataset, dataset["validation"])

  0%|          | 0/5036 [00:00<?, ?it/s]

{'exact_match': 57.744241461477365, 'f1': 78.08301491321691}

# Выводы:

Наша модель показывает хорошие результаты в задаче вопросно-ответных систем на этом наборе данных.
Она правильно ответила примерно на 58% вопросов и была близка к правильному ответу еще в ряде случаев, что подтверждается F1-оценкой в 78%.

Общие выводы:

- В коде явно не указано использование lr scheduler. Однако стоит отметить, что TrainingArguments в библиотеке transformers по умолчанию использует lr scheduler.
- В коде используется модель DeepPavlov/rubert-base-cased из библиотеки transformers, которая представляет собой одну из современных архитектур для QA. Это одна из вариаций BERT, предназначенная для создания многоязычных эмбеддингов предложений.
- В TrainingArguments указан параметр weight_decay, который является коэффициентом L2-регуляризации. Это помогает предотвратить переобучение модели.
- Используется стандартная функция потерь для задачи классификации токенов в AutoModelForTokenClassification. Эта функция потерь обычно основана на кросс-энтропии.